## This notebook + script uses astropy-healpix in order to discretize the sphere.

In [4]:
import numpy as np
from astropy import cosmology
import astropy_healpix
from astropy_healpix import HEALPix
import astropy.cosmology
from astropy.cosmology import Planck18 as cosmo
import astropy.units as unit
import argparse
import os
import pickle
from matplotlib import pyplot as plt

from pathlib import Path

f21 = 1.420405751e9 * unit.Hz


In [5]:
# Define frequency/redshift range being probed, as well as the number of shells.

start_freq = 1.6e8
end_freq = 1.8e8
Nfreqs = 32
start_freq *= unit.Hz
end_freq *= unit.Hz

freqs = np.linspace(start_freq, end_freq, Nfreqs)

redshifts = (f21 / freqs) - 1
z_order = np.argsort(redshifts)
f_order = np.argsort(freqs)
redshifts = redshifts[z_order]
redshifts = redshifts.to_value()
freqs = freqs[z_order]
min_redshift = np.min(redshifts)

In [6]:
# data cube of shape (pts_per_side, pts_per_side, pts_per_side), typically a 21cmFAST coeval cube
# but can also be, for instance, a user-generated np.random.normal(var = variance, size = (pts_per_side, pts_per_side, pts_per_side))
data_cube = np.load("z7_45_res100_len1000.npy")

## Algorithm for calculating the nearest point in the cube (and its value) for each point on a shell

In [7]:
# round_base(x, prec, base):
#    Rounds value x to the nearest integer multiple of base, with precision prec
def round_base(x, prec, base):     
    return (base * (np.array(x) / base).round()).round(prec)


#  nearest_cube_pt(x, y, z, cube_length, cube_res):
#      Input : 
#           x (float), y (float), z (float): coordinates of the shell point being mapped onto, units of [Mpc]
#           cube_length (int): comoving length of one side of the input data cube, units of [Mpc]
#           cube_res (int): number of points per side of the input data cube
#
#      Returns the nearest point of a cube with cube_length and cube_res
#      to point on the shell with coordinates (x, y, z).
# 
#      Assumes that cubes are uniformly tiled throughout the larger volume, such that (x y z) % cube_length is guaranteed
#      to give the coordinates of the input sphere point relative to the origin of the cube.

def nearest_cube_pt(x, y, z, cube_length, cube_res): 
    
    arr = np.array([x, y, z]) % cube_length
    nearest = round_base(arr, 5, cube_length / cube_res)
    nearest = (nearest * (cube_res / cube_length)).astype(int)
    
    # Imposes periodicity: if the index of any of the coordinates of the nearest cube point equals the max index + 1,
    #    set this index to 0, effectively "wrapping around" the cube
    nearest[nearest == cube_res] = 0
    
    return nearest

# cosm_plot(redshift, nside, cube_length, data_cube):
#      Input: 
#           redshift (float)
#           nside (int): desired shell resolution. Must be a power of 2, # of pixels on a shell = nside ^ 2 * 12
#           cube_length (int): comoving length of one side of the input data cube, units of [Mpc]
#           data_cube (numpy array, shape = (cube_res, cube_res, cube_res))
#
#      Returns a numpy array of the value of each point in the data cube nearest to each point on the shell, 
#           shape = (npix, 1)

def cosm_plot(redshift, nside, cube_length, data_cube):
    # cube_length in Mpc
    
    cube_res = data_cube.shape[0]
    
    hp = HEALPix(nside=nside)
    x, y, z = astropy_healpix.healpix_to_xyz(np.arange(hp.npix), nside = hp.nside)

    r_s = cosmology.Planck18.comoving_distance(redshift)
    xs = np.array(x * r_s)
    ys = np.array(y * r_s)
    zs = np.array(z * r_s)
    shell = np.array([xs, ys, zs]).T
    
    nearest_arr = nearest_cube_pt(xs, ys, zs, cube_length, cube_res)
    
    return data_cube[nearest_arr[0], nearest_arr[1], nearest_arr[2]]

## We run the algorithm on all shells across the redshift range of interest, returning an array of shape (nfreqs, npix = nside^2 * 12)

In [8]:

nside = 256
cube_len = 1000 # Mpc

mapped_vals = [cosm_plot(redshift, nside, cube_len, data_cube) for redshift in redshifts]